В задаче кластеризации нет целевой переменной. Вместо этого мы группируем локации на основе их сходства.

Кластеризация

Если задача заключается в группировке локаций по схожим характеристикам:

    K-Means:

        Простая и эффективная модель кластеризации, которая хорошо работает с данными, имеющими четкие кластеры.

    DBSCAN:

        Модель, которая может обнаруживать кластеры произвольной формы и хорошо работает с данными, содержащими шум.

    Иерархическая кластеризация (Hierarchical Clustering):

        Модель, которая строит иерархию кластеров и может быть полезна для визуализации структуры данных.

    Спектральная кластеризация (Spectral Clustering):

        Модель, которая хорошо работает с данными, имеющими сложную структуру, и может обнаруживать кластеры, которые не обязательно являются выпуклыми.

In [ ]:
import pandas as pd
import numpy as np
from pyproj import Transformer
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

In [ ]:
# Пример данных о локациях
data = {
    'latitude': [40.7128, 40.7306, 40.7484, 40.7580, 40.7688, 40.7788, 40.7888, 40.7988],
    'longitude': [-74.0060, -73.9352, -73.9857, -73.9857, -73.9681, -73.9581, -73.9481, -73.9381]
}

df = pd.DataFrame(data)

# Создаем трансформер для проекции Меркатора
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

# Преобразуем географические координаты в плоские
df['x'], df['y'] = transformer.transform(df['longitude'].values, df['latitude'].values)

# Функция для расчета манхэттенского расстояния
def manhattan_distance(x1, y1, x2, y2):
    return abs(x2 - x1) + abs(y2 - y1)

# Расчет матрицы манхэттенских расстояний
def compute_manhattan_distances(df):
    n = len(df)
    distances = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            distances[i, j] = manhattan_distance(df.loc[i, 'x'], df.loc[i, 'y'], df.loc[j, 'x'], df.loc[j, 'y'])
    return distances

# Вычисляем матрицу манхэттенских расстояний
distances_matrix = compute_manhattan_distances(df)

# Добавляем расстояния до всех локаций в качестве признаков
distances_df = pd.DataFrame(distances_matrix, columns=[f'distance_to_{j}' for j in range(len(df))])
df = pd.concat([df, distances_df], axis=1)

# Выбираем признаки для кластеризации
X = df.drop(columns=['latitude', 'longitude'])

# Кластеризация с использованием KMeans
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X)
labels_kmeans = kmeans.labels_

# Оценка качества кластеризации с использованием silhouette_score
silhouette_kmeans = silhouette_score(X, labels_kmeans)
print(f"Silhouette score для KMeans: {silhouette_kmeans}")

# Кластеризация с использованием AgglomerativeClustering
agg_clustering = AgglomerativeClustering(n_clusters=2)
labels_agg = agg_clustering.fit_predict(X)

# Оценка качества кластеризации с использованием silhouette_score
silhouette_agg = silhouette_score(X, labels_agg)
print(f"Silhouette score для AgglomerativeClustering: {silhouette_agg}")

# Визуализация кластеров
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.scatter(df['x'], df['y'], c=labels_kmeans, cmap='viridis')
plt.title('KMeans Clustering')

plt.subplot(1, 2, 2)
plt.scatter(df['x'], df['y'], c=labels_agg, cmap='viridis')
plt.title('Agglomerative Clustering')

plt.show()